In [ ]:
# Download CBDB sqlite
!git clone https://github.com/cbdb-project/cbdb_sqlite.git

Cloning into 'cbdb_sqlite'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 31 (delta 3), reused 9 (delta 3), pack-reused 22
Unpacking objects: 100% (31/31), done.


In [ ]:
# import glob
import os
latest_file = "./cbdb_sqlite/latest.7z"
latest_db = latest_file.split("/")[2].split(".")[0] + ".db"
print(latest_db)
if os.path.isfile(latest_db):
    os.remove(latest_db)

latest.db


In [ ]:
# Explode latest database file
!pip install pyunpack
!pip install patool
from pyunpack import Archive
Archive(latest_file).extractall(".")

     |████████████████████████████████| 77 kB 5.7 MB/s 


In [ ]:
# Create connection to database
import sqlite3
import pandas as pd
global CONN
database_file = os.path.basename(latest_file).split(".")[0]+".db"
CONN = sqlite3.connect(database_file)

In [ ]:
fullenames_df = pd.read_sql_query("SELECT c_name_chn FROM BIOG_MAIN", CONN).drop_duplicates().dropna()
# exclude c_alt_name_type_code = 7 (birth-order name / 行第, e.g. 第二十三)
givennames_df = pd.read_sql_query("SELECT c_alt_name_chn FROM ALTNAME_DATA where c_alt_name_type_code <> 7", CONN).drop_duplicates().dropna()

In [ ]:
def names_massage(df, column):
  # remove content in brackets
  df[column] = df[column].str.replace("[\(（].*[\)）]", "", regex=True)
  # remove single-character name
  df = df[df[column].apply(lambda x: len(x)>1)]
  # filter out weird characters
  df = df[df[column].str.contains('\?|？|#|!|！|【|。|、.|,|，|\*|、|\[|□|�|{|「')==False]
  df = df[df[column].str.contains('.*[a-zA-Z]+.*')==False]
  df = df[df[column].str.contains('\d')==False]
  df.drop_duplicates().dropna()
  return df

In [ ]:
filtered_fullnames = names_massage(fullenames_df, 'c_name_chn')
filtered_givennames = names_massage(givennames_df, 'c_alt_name_chn')

In [ ]:
filtered_fullnames

,c_name_chn
0,富興
1,富陞額
2,嵩山
3,得祿
4,恆善
...,...
491162,龔龍見
491164,郎侃
491165,郎崇賢
491166,郎汝璠


In [ ]:
filtered_givennames

,c_alt_name_chn
0,處厚
1,厚卿
2,湛然
3,元章
4,寳臣
...,...
139602,南阿山人
139606,明徵
139610,爾思
139611,飛虎將軍


In [ ]:
# combine df
final_df = pd.concat([filtered_fullnames, filtered_givennames], axis=1).stack().reset_index(drop=True).drop_duplicates().dropna().to_frame(name='name')

In [ ]:
final_df

,name
0,富興
1,處厚
2,富陞額
3,厚卿
4,嵩山
...,...
424961,龔龍見
424962,郎侃
424963,郎崇賢
424964,郎汝璠


In [ ]:
# Sort by descending order of the length
final_df = final_df.sort_values(by="name", key=lambda x: -x.str.len())
final_df

,name
132261,護國庇民妙靈昭應宏仁普濟福佑群生誠感咸孚顯神贊順垂慈篤祐天后
61535,承天廣運聖德神功肇紀立極仁孝睿武端毅欽安弘文定業高皇帝
167917,正倫立極盛德洪烈至誠大義格天熙運顯文毅武聖敬達孝大王
165301,受天興運敷化綏猷崇文經武光裕孝恭勤儉端敏英哲睿皇帝
165302,效天符運立中體正至文聖武智勇仁慈儉勤孝敏寬定成皇帝
...,...
100106,喬在
100105,習叟
100103,楓人
278882,田恪


In [ ]:
final_df.to_csv('cbdb_entity_names.csv', index=False)

In [ ]:
# close the db
CONN.close()